In [1]:
import csv
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

sk = []
kr = []
uid = []
peak = []
duration = []
freq = []
high_avg = []

df = pd.read_csv('training_set_0s_15.csv')
df = df.append(pd.read_csv('training_set_1s_15.csv'))
#print(df)

df = df.groupby('id', as_index=False, sort=False)['ev', 'skewness', 'kurtosis', 'mean_peak', 
                                                  'avg_high_energy_usage_duration', 
                                                  'high_energy_usage_frequency', 'mean_consumption'].mean()
#testing...
df.to_csv('aggregate_training_15min.csv')
uid = np.array(df['ev'].values)
sk = np.array(df['skewness'].values)
kr = np.array(df['kurtosis'].values)
peak = np.array(df['mean_peak'].values)
duration = np.array(df['avg_high_energy_usage_duration'].values)
freq = np.array(df['high_energy_usage_frequency'].values)
consumption_avg = np.array(df['mean_consumption'].values)

#normalization
#sk_norm = sk/np.linalg.norm(sk, ord=np.inf, axis=0, keepdims=True)
#kr_norm = kr/np.linalg.norm(kr, ord=np.inf, axis=0, keepdims=True)
#peak_norm = peak/np.linalg.norm(peak, ord=np.inf, axis=0, keepdims=True)
#duration_norm = duration/np.linalg.norm(duration, ord=np.inf, axis=0, keepdims=True)
#freq_norm = freq/np.linalg.norm(freq, ord=np.inf, axis=0, keepdims=True)
#high_avg_norm = high_avg/np.linalg.norm(high_avg, ord=np.inf, axis=0, keepdims=True)
#kr_norm = sklearn.preprocessing.normalize(kr)

X = []
with open('training_set_norm.csv', 'w') as fou:
    #w = csv.writer(fou, delimiter=',', lineterminator='\n')
    #w.writerow(['ev_id', 'skew_norm', 'kurtosis_norm', 'peak_norm', 'duration_norm', 'freq_norm', 'high_avg_norm'])
    #for i in range(0, len(uid)):
    #    X.append([sk_norm[i], kr_norm[i], high_avg_norm[i], peak_norm[i]])
    #    w.writerow([uid[i], sk_norm[i], kr_norm[i], peak_norm[i], duration_norm[i], freq_norm[i], high_avg_norm[i]])
    w = csv.writer(fou, delimiter=',', lineterminator='\n')
    w.writerow(['ev_id', 'skew_norm', 'kurtosis_norm', 'high_avg_norm', 'duration_norm', 'freq_norm', 'mean_consumption'])
    for i in range(0, len(uid)):
        X.append([sk[i], kr[i], peak[i], duration[i], freq[i], consumption_avg[i]])
        w.writerow([uid[i], sk[i], kr[i], peak[i], duration[i], freq[i], consumption_avg[i]])
#, peak_norm[i],duration_norm[i],freq_norm[i],high_avg_norm[i]        

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from itertools import product
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


Y = uid
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
#X_train = X
#X_test = X
#Y_train = Y
#Y_test = Y
from sklearn import preprocessing
from joblib import dump, load
scaler = preprocessing.StandardScaler().fit(X_train)
dump(scaler, 'scaler_15.joblib')
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)
#print(X_train_transformed, Y_train)
clf1 = SVC(gamma='scale', class_weight='balanced', kernel='rbf', shrinking=True, probability=True)
#print(Y)
clf1.fit(X_train_transformed, Y_train)
#print(clf.support_vectors_)
#print(clf.n_support_) 

dump(clf1, 'trained_ev_detect_svm_15.joblib') 
test_Y = clf1.predict(X_test_transformed)

#clf = LogisticRegression(random_state=0, solver='sag').fit(X, Y)
#test_X = clf.predict(X)
false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('svm') 
print(clf1.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

#clf2 = RandomForestClassifier(random_state=1)
clf3 = LogisticRegression(penalty='l2', solver='lbfgs', random_state=1, class_weight='balanced')
#eclf = VotingClassifier(estimators=[('svc', clf1), ('lr', clf3)], voting='soft', weights=[2, 1])
#clf2.fit(X, Y)
clf3.fit(X_train_transformed, Y_train)
dump(clf3, 'trained_ev_detect_lr_15.joblib') 
test_Y = clf3.predict(X_test_transformed)

#eclf.fit(X, Y)
#test_Y_el = eclf.predict(X)
false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('logregression') 
print(clf3.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)


clf2 = RandomForestClassifier(n_estimators=100, min_samples_split=6, min_samples_leaf=3, max_depth=50)

clf2.fit(X_train_transformed, Y_train)
test_Y = clf2.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('randomforest')     
print(clf2.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

dt_stump = DecisionTreeClassifier(max_depth=5, min_samples_leaf=10)
clf4 = AdaBoostClassifier(base_estimator=dt_stump)
clf4.fit(X_train_transformed, Y_train)
test_Y = clf4.predict(X_test_transformed)

false_positive = 0
false_negative = 0 
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('adaboost') 
print(clf4.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

from sklearn.ensemble import GradientBoostingClassifier
clf5 = GradientBoostingClassifier(n_estimators=100, min_samples_split=6, min_samples_leaf=3, 
                                  max_depth=20, random_state=0)
clf5.fit(X_train_transformed, Y_train)
test_Y = clf5.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('gradientboost')  
print(clf5.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

from sklearn.neighbors import KNeighborsClassifier
clf6 = KNeighborsClassifier(n_neighbors=2)
clf6.fit(X_train_transformed, Y_train)
test_Y = clf6.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('KNN')  
print(clf5.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

eclf = VotingClassifier(estimators=[('lr', clf3), ('gdb', clf5), ('rf', clf2)], 
                        voting='soft', weights=[1,2,1])
eclf.fit(X_train_transformed, Y_train)
test_Y = eclf.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
            
print(false_positive, true_negative, false_negative, true_positive)
print(eclf.score(X_test_transformed, Y_test))
score_proba = eclf.predict_proba(X_test_transformed)
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(Y_test, score_proba[:,1])
print(fpr)
print(tpr)
print(thresholds)
dump(eclf, 'trained_ev_detect_ensemble_15.joblib') 

svm
0.48148148148148145
8 9 6 4
logregression
0.6666666666666666
2 15 7 3
randomforest
0.6296296296296297
4 13 6 4
adaboost
0.48148148148148145
7 10 7 3
gradientboost
0.5925925925925926
5 12 6 4
KNN
0.5925925925925926
3 14 8 2
5 12 6 4
0.5925925925925926
[0.         0.         0.         0.11764706 0.11764706 0.29411765
 0.29411765 0.58823529 0.58823529 0.64705882 0.64705882 0.70588235
 0.70588235 0.76470588 0.76470588 0.88235294 0.88235294 0.94117647
 0.94117647 1.        ]
[0.  0.1 0.2 0.2 0.3 0.3 0.4 0.4 0.5 0.5 0.6 0.6 0.7 0.7 0.8 0.8 0.9 0.9
 1.  1. ]
[1.96031061 0.96031061 0.94920732 0.80144804 0.762809   0.61533893
 0.60524565 0.24809579 0.23592187 0.20368354 0.16453753 0.16425454
 0.15260869 0.1466947  0.14473603 0.1380256  0.1194044  0.08925049
 0.08722462 0.08006612]


['trained_ev_detect_ensemble_15.joblib']